In [1]:
import torch
import pandas as pd
from datetime import timedelta
from datetime import datetime

import numpy as np

from nera.data import *
from nera.models.ratings import EloGrad, EloManual, EloAutoGrad
from nera.reference import *


/home/kubos/anaconda3/envs/bp/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
da = DataAcquisition()
df = da.get_data(FROM_CSV, fname="../resources/other_leagues.csv")
df['DT'] = pd.to_datetime(df['DT'], format="%Y-%m-%d %H:%M:%S")
data_transform = DataTransformation(df, timedelta(365))
df = df[(df['League'] != 'EuroLeague') & (df['League'] != 'EuroCup')] 
df = df.reset_index()

transform = DataTransformation(df, timedelta(365))
dataset = transform.get_dataset(node_f_extract=False, edge_f_one_hot=True)

team_count = transform.num_teams

2024-02-29 13:37:40.875 | INFO     | nera.data._data_saving_loading:load_data_csv:70 - 21100 rows loaded from ../resources/other_leagues.csv


### Manual Elo with NN pass

In [3]:
# dummy dataset

delta = timedelta(days=365)
delta2 = timedelta(days=366)
now = datetime.now()
data = pd.DataFrame({'DT': [*(3 * [now]), *(3 * [now - delta2]), *(3 * [now - 2 * delta2]), *(3 * [now - 3 * delta2])], 
                     'Home': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                     'Away': ['B', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'C', 'A', 'B'],
                     'Winner': ['home', 'away', 'away', 'home', 'away', 'away', 'home', 'home', 'away', 'home', 'away', 'home'],
                     'Home_points': [10, 5, 15, 12, 15, 6, 20, 10, 10, 14, 3, 12],
                     'Away_points': [4, 8, 17, 10, 16, 14, 18, 9, 15, 0, 11, 4],
                     'League': [*(12 * ['liga'])],
                     })

transform2 = DataTransformation(data, timedelta(days=365))
dts = transform2.get_dataset(edge_f_one_hot=True)

In [4]:

def train_elo_manual(train_dataset, model, epochs: int = 100, verbose: bool = False):
    torch.set_printoptions(precision=8) 
    training_accuracy = []
    loss_fn = torch.nn.MSELoss
    
    E_HS = []
    elos = []
    pts = []
    
    for epoch in range(epochs):
        model.train()
        accuracy, loss, count = 0, 0, 0
        iter = 0
        for time, snapshot in enumerate(train_dataset):
            # pass through network has to be only one by one in order to compute elo correctly
            matches = snapshot.edge_index
            match_points = snapshot.match_points
            for m in range(matches.shape[1]):
                match = matches[:, m]
                
                y_hat = model(match)
                y = snapshot.edge_attr[m, :]  # edge weight encodes the match outcome
                
                E_HS.append(y_hat)
                
                elos_b4 = (model.rating[model.home], model.rating[model.away])
                  
                target = torch.argmax(y) / 2.
                target = target.detach()
                prediction = y_hat
                accuracy += 1 if abs(target - prediction) < 0.5 else 0
    
                loss += torch.mean((prediction-target)**2)
                
                point_diff = torch.abs(match_points[m, 0] - match_points[m, 1]).detach()
                result = [target, point_diff]
                
                model.backward(result)
                
                elos.append((elos_b4, (model.rating[model.home], model.rating[model.away])))
                pts.append(((match_points[m, 0], match_points[m, 1]), (model.home, model.away)))
                
                # if verbose:
                #    rating = model.rating[:5] if len(model.rating) >= 5 else model.rating
                #    print(f"Neuralnet {iter}, rating: {rating}, E_H = {y_hat}; {target}")
                iter += 1
                
            count += matches.shape[1]

        if verbose:
            rating = model.rating[:5] if len(model.rating) >= 5 else model.rating
            print(f'[TRN] '
                  f' Epoch: {epoch}, training loss: {loss:.3f}, '
                        f'training accuracy: {accuracy/count * 100:.2f}% \n'
                        f'ratings (first 5): {rating}')
        training_accuracy.append(accuracy/count * 100)
    
    return np.array(training_accuracy), E_HS, elos, pts

    

In [5]:
reference_maker = RatingReference(data, transform2.team_mapping)

print('Computed elo Dummy: ', reference_maker.compute_reference('elo')[0])
eloDummy = EloManual(team_count=transform2.num_teams)
acc_dummy = train_elo_manual(dts, eloDummy, 1, True)

print()
print('----------------------------------------')
print()

reference_maker.matches = (df, transform.team_mapping)
computed, E_HS_comp, elos_comp = reference_maker.compute_reference('elo')

print()
elo = EloManual(team_count=transform.num_teams)
acc_late, E_HS_net, elos_net, pts = train_elo_manual(dataset, elo, 1, verbose=True)

err = False
for i in range(len(computed)):
    cmp = float(computed[transform.team_mapping[transform.inv_team_mapping[i]]])
    net = float(elo.rating[transform.team_mapping[transform.inv_team_mapping[i]]])
    if cmp - net > 0.1:
        print(rf'ERROR on index {i}:: {cmp} / {net}')
        err = True
    if i < 5:
        print(f'{i}:: computed: {cmp:10.3f} || net: {net:10.3f}')
if not err:
    print("...")
    print("[SUCCESS]: Computed elo is the same as Elo from NN")

print_diff = False
if print_diff: 
    inv_map = {v: k for k, v in transform.team_mapping.items()}
    for i in range(len(E_HS_net)):
        eq = E_HS_net[i] == E_HS_comp[i]
        
        str_i = f'{i}:: net: {E_HS_net[i]:2.2f}, comp: {E_HS_comp[i]:2.2f};;; elo net / comp: {elos_net[i][1][0]:10.2f} / {elos_comp[i][1][0]:10.2f} :: {elos_net[i][1][1]:10.2f} / {elos_comp[i][1][1]:10.2f}'
        
        print(str_i if eq else '----------> ' + str_i + f''' {df.iloc[i]["Home"]}  home_pts: {df.iloc[i]["Home_points"]}, {df.iloc[i]["Away"]}  away_pts: {df.iloc[i]["Away_points"]};;;;;;;;; pts net: {pts[i][0][0]}, {pts[i][0][1]}''')
        if not eq:
            print(f'elo before: {elos_net[i][0][0]:10.2f} / {elos_comp[i][0][0]:10.2f} :: {elos_net[i][0][1]:10.2f} / {elos_comp[i][0][1]:10.2f}')
            print(f'Teams: home {inv_map[int(pts[i][1][0])]} away {inv_map[int(pts[i][1][1])]}')
            break 


Computed elo Dummy:  [1450.41466185  783.32864878  766.25668937]
[TRN]  Epoch: 0, training loss: 2.543, training accuracy: 75.00% 
ratings (first 5): Parameter containing:
tensor([1450.41466185,  783.32864878,  766.25668937], dtype=torch.float64,
       requires_grad=True)

----------------------------------------
[TRN]  Epoch: 0, training loss: 6166.810, training accuracy: 53.27% 
ratings (first 5): tensor([1379.68811002, 2976.96673204,  384.48140654,  571.46967020,
        1309.88250176], dtype=torch.float64, grad_fn=<SliceBackward0>)
0:: computed:   1379.688 || net:   1379.688
1:: computed:   2976.967 || net:   2976.967
2:: computed:    384.481 || net:    384.481
3:: computed:    571.470 || net:    571.470
4:: computed:   1309.883 || net:   1309.883
...
[SUCCESS]: Computed elo is the same as Elo from NN


### Elo with gradient

first lets check our analytical backward pass

In [8]:
from torch.autograd import gradcheck
from nera.models.ratings._elo._grad import elo_function
# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True))
test = gradcheck(elo_function, input, eps=1e-6, atol=1e-4)
print(test)

True


Define training function for elo

In [9]:
from nera.models.loss import WeightedMSELoss
from torch.optim import Adam

def train_elo_grad(train_dataset, model, epochs: int = 1, verbose: bool = False, lr:float = -1, clip_grad: bool = False, debug=False):
    torch.set_printoptions(precision=8) 
    training_accuracy = []
    loss_fn = WeightedMSELoss()
    
    lr = model.k if lr == -1 else lr
    
    optim = Adam(model.parameters(), lr=lr)
    
    E_HS = []
    elos = []
    pts = []
    
    for epoch in range(epochs):
        model.train()
        accuracy, loss, count = 0, 0, 0
        iter = 0
        for time, snapshot in enumerate(train_dataset):
            # pass through network has to be only one by one in order to compute elo correctly
            matches = snapshot.edge_index
            match_points = snapshot.match_points
            optim.zero_grad()
            for m in range(matches.shape[1]):
                last_c, last_d = model.c, model.d
                
                match = matches[:, m]
                
                y_hat = model(match)
                y = snapshot.edge_attr[m, :]  # edge weight encodes the match outcome
                y.requires_grad = True
                E_HS.append(y_hat)
                
                elos_b4 = (model.rating[model.home], model.rating[model.away])
                  
                target = torch.argmax(y) / 2.
                target = target.detach()
                prediction = y_hat
                accuracy += 1 if abs(target - prediction) <= 0.5 else 0
                
                point_diff = torch.abs(match_points[m, 0] - match_points[m, 1])
                loss = loss_fn(y, y_hat, (point_diff + 1) ** model.gamma)
                
                loss.backward()
                
                if clip_grad:
                    # Clip gradients to prevent explosion
                    # This should be used when training c, d hyper params as well
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  
        
                optim.step()
                
                with torch.no_grad():
                    model.c.clamp_(min=0)
                    if float(model.c) - 1e-6 < 0:
                        model.c += 1e-2
                 
                if debug:
                    print(f"c: {float(model.c)}, d: {float(model.d)}")
                    if torch.isnan(model.c).item() or torch.isnan(model.d).item():
                        return 0,0,0,0
                
                elos.append((elos_b4, (model.rating[model.home], model.rating[model.away])))
                pts.append(((match_points[m, 0], match_points[m, 1]), (model.home, model.away)))
                
                # if verbose:
                #    rating = model.rating[:5] if len(model.rating) >= 5 else model.rating
                #    print(f"Neuralnet {iter}, rating: {rating}, E_H = {y_hat}; {target}")
                iter += 1
                
            count += matches.shape[1]

        if verbose:
            rating = model.rating[:5] if len(model.rating) >= 5 else model.rating
            print(f'[TRN] '
                  f' Epoch: {epoch}, training loss: {loss:.3f}, '
                        f'training accuracy: {accuracy/count * 100:.2f}% \n'
                        f'ratings (first 5): {rating}')
        training_accuracy.append(accuracy/count * 100)
    
    return np.array(training_accuracy), E_HS, elos, pts



Now, let's train elo with analytical backward pass

In [10]:
elo_grad = EloGrad(team_count, cd_grad=False, c=torch.tensor(3, dtype=torch.float64), d=torch.tensor(500., dtype=torch.float64))
acc_grad, ehs_grad, elos_grad, pts_grad = train_elo_grad(dataset, elo_grad, clip_grad=False, verbose=True, lr=3.)

[TRN]  Epoch: 0, training loss: 55.045, training accuracy: 47.81% 
ratings (first 5): tensor([ 2378.96681058, 15953.91269562, -7156.40087564, -9066.71108653,
         3766.26643145], dtype=torch.float64, grad_fn=<SliceBackward0>)


And Elo with autograd (numerical gradient backward pass)

In [11]:
elo_auto = EloAutoGrad(team_count, cd_grad=False, c=torch.tensor(3, dtype=torch.float64), d=torch.tensor(500., dtype=torch.float64))
acc_auto, ehs_auto, elos_auto, pts_auto = train_elo_grad(dataset, elo_auto, clip_grad=False, verbose=True, lr=3.)

[TRN]  Epoch: 0, training loss: 55.045, training accuracy: 47.81% 
ratings (first 5): tensor([ 2378.96681058, 15953.91347711, -7156.44467282, -9066.71814052,
         3766.09355290], dtype=torch.float64, grad_fn=<SliceBackward0>)


And finally, compare wheter these two are the same
___________________

In [12]:
def print_diff(numerical, analytical, eps: float = 1e-2):
    err = False
    err_sum = 0
    err_count = 0
    print_count = 0
    max_diff = 0
    for i in range(len(numerical.rating)):
        auto = float(numerical.rating[transform.team_mapping[transform.inv_team_mapping[i]]])
        grad = float(analytical.rating[transform.team_mapping[transform.inv_team_mapping[i]]])
        diff = abs(auto - grad)
        if diff > eps:
            err = True
            err_sum += diff
            err_count += 1
            max_diff = max(diff, max_diff)
        if print_count < 5:
            str_i = rf'[ERROR] on index {i}:: {auto} / {grad}' if err else f'{i}:: computed: {auto:10.3f} || net: {grad:10.3f}'
            print(str_i)
            print_count += 1
    
    print("...")
    print()
    
    if not err:
        print("[SUCCESS]: All elo ratings computed analytically and numerically are the SAME")
    else:
        print(f"Total number of errors: {err_count} out of {len(numerical.rating)} computed ratings")
        print(f"Cumulative sum of errors: {err_sum}")
        print(f"Average difference: {err_sum / err_count}")
        print(f"Maximal difference: {max_diff}")
        
    print()
        
    print(f"Numerical:  c = {float(numerical.c):8.3f} :: d = {float(numerical.d):8.3f}")
    print(f"Analytical: c = {float(analytical.c):8.3f} :: d = {float(analytical.d):8.3f}")

In [13]:
print_diff(numerical=elo_auto, analytical=elo_grad)

0:: computed:   2378.967 || net:   2378.967
1:: computed:  15953.913 || net:  15953.913
[ERROR] on index 2:: -7156.444672822601 / -7156.400875638662
[ERROR] on index 3:: -9066.718140522222 / -9066.711086527102
[ERROR] on index 4:: 3766.093552898145 / 3766.2664314509893
...

Total number of errors: 53 out of 146 computed ratings
Cumulative sum of errors: 58.48463412058629
Average difference: 1.1034836626525715
Maximal difference: 24.77435875706351

Numerical:  c =    3.000 :: d =  500.000
Analytical: c =    3.000 :: d =  500.000


Now let's see what happens, when we let the model learn the c, d metaparameters as well

In [14]:
elo_auto2 = EloAutoGrad(team_count, hp_grad=True, c=torch.tensor(5000, dtype=torch.float64), d=torch.tensor(50000, dtype=torch.float64))
acc_auto2, ehs_auto2, elos_auto2, pts_auto2 = train_elo_grad(dataset, elo_auto2, clip_grad=True, verbose=True, epochs=1, debug=False, lr=10)

[TRN]  Epoch: 0, training loss: 40.595, training accuracy: 46.52% 
ratings (first 5): tensor([-2745.78053601,  7549.90160180,   187.87634065, -4116.20747451,
        12916.09422653], dtype=torch.float64, grad_fn=<SliceBackward0>)


In [15]:
elo_grad2 = EloGrad(team_count, hp_grad=True, c=torch.tensor(5000., dtype=torch.float64), d=torch.tensor(50000, dtype=torch.float64))
acc_grad2, ehs_grad2, elos_grad2, pts_grad2 = train_elo_grad(dataset, elo_grad2, lr=10, clip_grad=True, verbose=True)

[TRN]  Epoch: 0, training loss: 40.595, training accuracy: 46.52% 
ratings (first 5): tensor([-2745.78053601,  7549.90160179,   187.87634065, -4116.20747451,
        12916.09422653], dtype=torch.float64, grad_fn=<SliceBackward0>)


In [16]:
print_diff(numerical=elo_auto2, analytical=elo_grad2)

0:: computed:  -2745.781 || net:  -2745.781
1:: computed:   7549.902 || net:   7549.902
2:: computed:    187.876 || net:    187.876
3:: computed:  -4116.207 || net:  -4116.207
4:: computed:  12916.094 || net:  12916.094
...

[SUCCESS]: All elo ratings computed analytically and numerically are the SAME

Numerical:  c = 11677.763 :: d = 191906.782
Analytical: c = 11677.763 :: d = 191906.782
